In [1877]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import time
import pytz
import dateutil
from dateutil import relativedelta

sns.set(color_codes=True)
sns.set(rc={"figure.figsize": (16, 6)})

In [1878]:
# Database connection credentials

user = "postgres"
password = "200323201"
host = "localhost"
port = "5432"
database = "heroku-timesheet"

In [1879]:
db_string = "postgresql://{user}:{password}@{host}:{port}/{database}".format(user=user, 
                                                                             password=password,
                                                                             host=host,
                                                                             port=port,
                                                                             database=database)

In [1880]:
from sqlalchemy import create_engine
engine = create_engine(db_string)
con = engine.connect()

In [1881]:
%%html
<style>
table { float: left }
</style>

<style>
table {float:left}
</style>

| data | source | formula | remarks |
| ---------- | ------ | ------- | ------- |
| orgname | organization.name |  |  |
| since | timesheet.start_datetime | min | to be convert to date string |
| timdelta | timesheet.start_datetime | min duration | to be convert to timedelta string |
| people_count | timesheet.person_id | count distinct | billed members ever |
| project_count | timesheet.project_id | count distinct | billed project ever |
| active_project_count | timesheet.project_id | count distinct / <30 days | billed project in recent 30 days, not base on status |
| total_hours | timesheet.duration | sum in hours | total billed hours |
| total_eng_months | timesheet.duration | sum in months | total billed hours in months |
| total_gross | timesheet.total | sum | total income |
| total_investment | timesheet.total_discount | sum | total discount |

In [1882]:
query = """
    SELECT model.organization.name,
           model.timesheet.start_datetime,
           model.timesheet.person_id,
           model.timesheet.project_id,
           model.timesheet.duration,
           model.timesheet.total,
           model.timesheet.total_discount
    FROM model.organization
    LEFT JOIN model.timesheet
           ON model.organization.id = model.timesheet.organization_id
        """

In [1883]:
timesheet = pd.read_sql(query, con)

In [1884]:
#What time is it? The original time should have considered timezone?
#Can ignore this. Just some experiments on conversion across timezone

test0 = timesheet['start_datetime'][0]
display(test0)

test1 = test0.astimezone(pytz.timezone('Asia/Shanghai'))
display(test1)

test2 = test0.replace(tzinfo = pytz.timezone('Asia/Shanghai'))
display(test2)

test3 = test0.replace(tzinfo = pytz.timezone('Asia/Shanghai')).astimezone(pytz.UTC)
display(test3)

Timestamp('2017-09-20 01:00:00+0000', tz='UTC')

Timestamp('2017-09-20 09:00:00+0800', tz='Asia/Shanghai')

Timestamp('2017-09-20 01:00:00+0806', tz='Asia/Shanghai')

Timestamp('2017-09-19 16:54:00+0000', tz='UTC')

In [1885]:
timesheet['timedelta'] = now - timesheet['start_datetime']

In [1886]:
#from timedelta to relativedelta
#relativedelta.relativedelta(seconds = timedelta.total_seconds())

In [1887]:
#how to convert timedelta/relativedelta to "? Years, ? Months and ? Days" elegantly?
#how to solve the problem that relativedelta does not support aggregation?
#how to handle timestamp/datetime errors i.e. NaT? Other error like None?
#what is the difference between timedelta and relativedelta?
#what is the use of relativedetla?

In [1888]:
#create now() timestamp with timezone UTC
now = pytz.UTC.localize(datetime.datetime.utcnow())

In [1889]:
def diff_relativedelta(timestamp):
    
    if timestamp != 'NaT':
        return relativedelta.relativedelta(now, timestamp)
    else:
        return relativedelta.relativedelta(0)

In [1890]:
timesheet.iloc[4093, 1] = 'NaT'
timesheet['relativedelta'] = timesheet['start_datetime'].apply(lambda x: diff_relativedelta(x).years)

In [1891]:
timesheet.tail()

,name,start_datetime,person_id,project_id,duration,total,total_discount,timedelta,relativedelta
4089,Coderbunker Shanghai,2017-11-08 10:00:00+00:00,17d8cefa-c01d-4ec5-85b5-1c5b2edc533c,7446253a-af9e-40bf-ba60-e1d42117d7d3,00:30:00,0.0,350.0,173 days 18:12:57.200531,0
4090,Coderbunker Shanghai,2017-11-08 11:30:00+00:00,17d8cefa-c01d-4ec5-85b5-1c5b2edc533c,7446253a-af9e-40bf-ba60-e1d42117d7d3,01:30:00,0.0,1050.0,173 days 16:42:57.200531,0
4091,Coderbunker Shanghai,2017-10-24 06:00:00+00:00,17d8cefa-c01d-4ec5-85b5-1c5b2edc533c,7446253a-af9e-40bf-ba60-e1d42117d7d3,03:00:00,0.0,2100.0,188 days 22:12:57.200531,0
4092,Coderbunker Shanghai,2017-10-26 02:00:00+00:00,17d8cefa-c01d-4ec5-85b5-1c5b2edc533c,7446253a-af9e-40bf-ba60-e1d42117d7d3,02:00:00,0.0,1400.0,187 days 02:12:57.200531,0
4093,Coderbunker Singapore,NaT,None,None,NaT,NaN,NaN,NaT,0


In [1892]:
timesheet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4094 entries, 0 to 4093
Data columns (total 9 columns):
name              4094 non-null object
start_datetime    4094 non-null object
person_id         4093 non-null object
project_id        4093 non-null object
duration          4093 non-null timedelta64[ns]
total             4093 non-null float64
total_discount    4093 non-null float64
timedelta         4093 non-null timedelta64[ns]
relativedelta     4094 non-null int64
dtypes: float64(2), int64(1), object(4), timedelta64[ns](2)
memory usage: 287.9+ KB


In [1893]:
#getting every values using groupby, then concate into a full table

since = timesheet.groupby('name')['start_datetime'].min().reset_index().rename(columns={'start_datetime': 'since'})
since

,name,since
0,Coderbunker Shanghai,2016-10-17 02:00:00+00:00
1,Coderbunker Singapore,NaT


In [1894]:
duration = timesheet.groupby('name')['timedelta'].max().reset_index()
duration

,name,timedelta
0,Coderbunker Shanghai,561 days 02:12:57.200531
1,Coderbunker Singapore,NaT


In [1895]:
duration_relativedelta = timesheet.groupby('name')['relativedelta'].max().reset_index()
duration_relativedelta

,name,relativedelta
0,Coderbunker Shanghai,1
1,Coderbunker Singapore,0


In [1896]:
people_count = timesheet.groupby('name')['person_id'].nunique().reset_index().rename(columns={'person_id': 'people_count'})
people_count

,name,people_count
0,Coderbunker Shanghai,54
1,Coderbunker Singapore,0


In [1897]:
project_count = timesheet.groupby('name')['project_id'].nunique().reset_index().rename(columns={'project_id': 'project_count'})
project_count

,name,project_count
0,Coderbunker Shanghai,19
1,Coderbunker Singapore,0


In [1898]:
active_project_count_old = timesheet[timesheet['timedelta'] < datetime.timedelta(days = 30)]
active_project_count_old = active_project_count_old.groupby('name')['project_id'].nunique().reset_index().rename(columns={'project_id': 'active_project_count'})
active_project_count_old

,name,active_project_count
0,Coderbunker Shanghai,9


In [1899]:
#How to return 0/None for Coderbunker Singapore too?
#Create a function to count active project...?

def count_active_project(timesheet):
    active_project_id = []
    for x, y in zip(timesheet['project_id'], timesheet['timedelta']):
        if y < datetime.timedelta(days = 30):
            active_project_id.append(x)
    return(len(set(active_project_id)))

In [1900]:
active_project_count = timesheet.groupby('name').apply(count_active_project).reset_index().rename(columns={0: 'active_project_count'})
active_project_count

,name,active_project_count
0,Coderbunker Shanghai,9
1,Coderbunker Singapore,0


In [1901]:
total_hours = timesheet.groupby('name')['duration'].sum().reset_index().rename(columns={'duration': 'total_hours'})
total_hours['total_hours'] = total_hours['total_hours'].apply(lambda x: round(x.total_seconds()/3600))
total_hours

,name,total_hours
0,Coderbunker Shanghai,9077
1,Coderbunker Singapore,0


In [1902]:
total_eng_months = total_hours.rename(columns={'total_hours':'total_eng_months'})
total_eng_months['total_eng_months'] = total_eng_months['total_eng_months'].apply(lambda x: round(x/168))
total_eng_months

,name,total_eng_months
0,Coderbunker Shanghai,54
1,Coderbunker Singapore,0


In [1903]:
total_gross = timesheet.groupby('name')['total'].sum().reset_index().rename(columns={'total': 'total_gross'})
total_gross['total_gross'] = total_gross['total_gross'].apply(lambda x: round(x))
total_gross

,name,total_gross
0,Coderbunker Shanghai,2005301
1,Coderbunker Singapore,0


In [1904]:
total_investment = timesheet.groupby('name')['total_discount'].sum().reset_index().rename(columns={'total_discount': 'total_investment'})
total_investment['total_investment'] = total_investment['total_investment'].apply(lambda x: round(x))
total_investment

,name,total_investment
0,Coderbunker Shanghai,200020
1,Coderbunker Singapore,0


In [1905]:
frames = [since, duration, people_count, project_count, active_project_count, total_hours, total_eng_months, total_gross, total_investment]
organization = pd.concat(frames, axis = 1, join = 'inner').T.drop_duplicates().T
organization = organization.rename({'name': 'orgname'}, axis='columns').set_index('orgname').T
organization

orgname,Coderbunker Shanghai,Coderbunker Singapore
since,2016-10-17 02:00:00+00:00,NaT
timedelta,561 days 02:12:57.200531,NaT
people_count,54,0
project_count,19,0
active_project_count,9,0
total_hours,9077,0
total_gross,2005301,0
total_investment,200020,0
